# 🪄 Optimize models for the ONNX Runtime
### Task: Text Generation 📝

In this notebook, you'll:

1. Optimize Small Language Model(s) (SLMs) from a curated list. Model *architectures* include: Qwen, Llama, Phi, Gemma and Mistral.
1. Inference the optimized SLM on the ONNX Runtime as part of a simple console chat application.


## 🐍 Python dependencies

Install the following Python dependencies:

In [ ]:
%%capture

%pip install olive-ai[auto-opt]
%pip install transformers==4.44.2 onnxruntime-genai

## ☑️ Select an SLM

Select a model from the list below by uncommenting your model of choice and ensuring all other models are commented out.

The list below is **not** an exhaustive list of text generation models supported by Olive and the ONNX Runtime. Instead, we have curated this list of models that are:

- *"small"* (less than ~7B parameters) and 
- *"popular"* (i.e. either high trending/download/liked models on Hugging Face)

You can optimize and inference other Generative AI models from the following model *architectures* using this notebook:

1. Qwen
1. Llama (includes Smol)
1. Phi
1. Gemma
1. Mistral

Other model architectures are also supported by Olive and the ONNX Runtime - for example [opt-125m](https://github.com/microsoft/Olive/tree/main/examples/opt_125m) and [falcon](https://github.com/microsoft/Olive/tree/main/examples/falcon) - However, they are are not yet supported in the ONNX Runtime Generate API and therefore require you to inference using lower-level APIs.

In [2]:
# ======================= QWEN MODELS ===========================
# MODEL="Qwen/Qwen2.5-0.5B-Instruct"
MODEL="Qwen/Qwen2.5-1.5B-Instruct"
# MODEL="Qwen/Qwen2.5-3B-Instruct"
# MODEL="Qwen/Qwen2.5-7B-Instruct"
# MODEL="Qwen/Qwen2.5-Math-1.5B-Instruct"
# MODEL="Qwen/Qwen2.5-Coder-7B-Instruct"
#================================================================

# ======================= LLAMA MODELS ==========================
# MODEL="meta-llama/Llama-3.2-1B-Instruct"
# MODEL="meta-llama/Llama-3.2-3B-Instruct"
# MODEL="meta-llama/Meta-Llama-3-8B-Instruct"
# MODEL="meta-llama/CodeLlama-7b-hf"
# MODEL="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#================================================================

# ======================= PHI MODELS ============================
# MODEL="microsoft/Phi-3.5-mini-instruct"
# MODEL="microsoft/Phi-3-mini-128k-instruct"
# MODEL="microsoft/Phi-3-mini-4k-instruct"
#================================================================

# ======================= SMOLLM2 MODELS ========================
# MODEL="HuggingFaceTB/SmolLM2-135M-Instruct"
# MODEL="HuggingFaceTB/SmolLM2-360M-Instruct"
# MODEL="HuggingFaceTB/SmolLM2-1.7B-Instruct"
#================================================================

# ======================= GEMMA MODELS ==========================
# MODEL="google/gemma-2-2b-it"
# MODEL="google/gemma-2-9b-it"
#================================================================

# ======================= MISTRAL MODELS ========================
# MODEL="mistralai/Ministral-8B-Instruct-2410"
# MODEL="mistralai/Mistral-7B-Instruct-v0.3"
#================================================================


### 🤗 Login to Hugging Face
To access models, you'll need to log-in to Hugging Face with a [user access token](https://huggingface.co/docs/hub/security-tokens). The following command will run you through the steps to login:

In [ ]:
!huggingface-cli login

### 📇 Model card

The code in the following cell gets some information on the selected model (such as license and number of downloads)

In [ ]:
import huggingface_hub as hf

m=hf.repo_info(MODEL)
print(f"Model Card :https://huggingface.co/{MODEL}")
print(f"License: {m.card_data['license']}, {m.card_data['license_link']}")
print(f"Number of downloads: {m.downloads}")
print(f"Number of likes: {m.likes}")


## 🪄 Run the Auto Optimizer

Next, you'll execute Olive's automatic optimizer using the optimize CLI command, which will:

1. Acquire the model from the the Hugging Face model repo.
1. Quantize the model to `int4` using GPTQ.
1. Capture the ONNX Graph and store the weights in an ONNX data file.
1. Optimize the ONNX Graph.

In [ ]:
!olive optimize \
    --model_name_or_path {MODEL} \
    --output_path models/{MODEL} \
    --precision int4

## 🧠 Inference model using ONNX Runtime

The ONNX Runtime (ORT) is a fast and light-weight package (available in many programming languages) that runs cross-platform. ORT enables you to infuse your AI models into your applications so that inference is handled *on-device*. The following code creates a simple console-based chat interface that inferences your optimized model.

### How to use

The sample chat app to run is found as [model-chat.py](https://github.com/microsoft/onnxruntime-genai/blob/main/examples/python/model-chat.py) in the [onnxruntime-genai](https://github.com/microsoft/onnxruntime-genai/) Github repository.